## Installed packages

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib
import io
import urllib
import tempfile
import functools
import logging
import pickle
import opentnsim
import yaml
import math
from osgeo import gdal, ogr

# package(s) related to the simulation
import enum
import simpy
import scipy as sc
import math
import networkx as nx  
import numpy as np
import pandas as pd
import re
import yaml as yaml
import bisect
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.tri as tri

from dataclasses import dataclass
from enum import Enum
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from matplotlib import cm

# OpenTNSim
from opentnsim import core
from opentnsim import plot
from opentnsim import model

# spatial libraries 
import shapely.geometry
import shapely.wkt
import pyproj
import shapely.geometry
from osgeo import ogr, osr
from simplekml import Kml, Style
import folium
import time as timepy

# package(s) for data handling
import requests
import math             
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# define the coorinate system
geod = pyproj.Geod(ellps="WGS84")

location_vessel_database = "03_vessels/richtlijnen-vaarwegen-2017.csv"

## Some functions

In [2]:
@functools.lru_cache
def load_fis_network(url):
    """load the topological fairway information system network (vaarweginformatie.nl)"""

    # get the data from the url
    resp = requests.get(url)
    # convert to file object
    stream = io.StringIO(resp.text)
    
    # This will take a minute or two
    # Here we convert the network to a networkx object
    G = yaml.load(stream, Loader=yaml.Loader)

    # some brief info
    n_bytes = len(resp.content)
    msg = '''Loaded network from {url} file size {mb:.2f}MB. Network has {n_nodes} nodes and {n_edges} edges.'''
    summary = msg.format(url=url, mb=n_bytes / 1000**2, n_edges=len(G.edges), n_nodes=len(G.nodes))
    logger.info(summary)

    # The topological network contains information about the original geometry. 
    # Let's convert those into python shapely objects for easier use later
    for n in G.nodes:
        G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
    for e in G.edges:
        edge = G.edges[e]
        edge['geometry'] = shapely.wkt.loads(edge['Wkt'])
        edge['length'] = edge_length(edge)    
    
    return G 

def edge_length(edge):
    """compute the great circle length of an edge
    The network version 0.1 contains the lat/lon distance in a length property. 
    But we need the "great circle" or projected distance. 
    Let's define a function to recompute it.
    """
    
    # get the geometry
    geom = edge['geometry']
    # get lon, lat
    lats, lons = np.array(geom).T
    # this requires pyproj 2.3.0
    distance = geod.line_length(lons, lats)

    return distance

def find_closest_edge(G, point):
    """find the closest edge on the graph from a given point"""
    
    distance = np.full((len(G.edges)), fill_value=np.nan)
    for ii, e in enumerate(G.edges):
        distance[ii] = point.distance(G.edges[e]['Info']['geometry'])
    name_edge = list(G.edges)[np.argmin(distance)]
    distance_edge = np.min(distance)
    
    return name_edge, distance_edge

def find_closest_node(G, point):
    """find the closest node on the graph from a given point"""
    
    distance = np.full((len(G.nodes)), fill_value=np.nan)
    for ii, n in enumerate(G.nodes):
        distance[ii] = point.distance(G.nodes[n]['geometry'])
    name_node = list(G.nodes)[np.argmin(distance)]
    distance_node = np.min(distance)
    
    return name_node, distance_node

## Loading hydrodata sets

In [3]:
file_path = []
file_path.append(os.path.join(os.getcwd(),'WL_dataset.csv'))
file_path.append(os.path.join(os.getcwd(),'VM_dataset.csv'))
file_path.append(os.path.join(os.getcwd(),'Van_dataset.csv'))

df_wlevel= pd.read_csv(file_path[0],delimiter=',')
df_VM= pd.read_csv(file_path[1],delimiter=',')
df_Van= pd.read_csv(file_path[2],delimiter=',')

## Import FIS network

In [4]:
url = 'https://zenodo.org/record/4578289/files/network_digital_twin_v0.2.yaml'

# create a cached version to speed up loading (remove cached file if a better yaml file is available)
fname = "fis_cache\\{}.pkl".format('FIS')

if os.path.exists(fname):
    print('I am loading cached network')
    with open(fname, 'rb') as pkl_file:
        FG = pickle.load(pkl_file)
        pkl_file.close()

else:
    print('I am getting new network')
    FG = load_fis_network(url)

    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, 'wb') as pkl_file:
        pickle.dump(FG, pkl_file)
        pkl_file.close()

I am loading cached network


### Selection of edges and nodes within the Port of Rotterdam

In [5]:
file_path1 = os.path.join(os.getcwd(),'02_hydrodynamic_data','Polygon FIS Network PoR.kml')
file_path1 = os.path.join(os.getcwd(),'02_hydrodynamic_data','Polygon FIS Network PoR.json')

srcDS = gdal.OpenEx(file_path1)
gdal.VectorTranslate('Polygon FIS Network PoR.json', srcDS, format='GeoJSON')

polygon = shapely.geometry.Polygon([[4.047046106288641, 51.994530949601149, 0.0 ], [ 4.041820334307678, 51.985794447379547, 0.0 ], [ 4.032638009410599, 51.987359301430168, 0.0 ], [ 4.026085029365865, 51.986529814588927, 0.0 ], [ 4.023082985071111, 51.985502004661448, 0.0 ], [ 4.00823290473104, 51.988174970713708, 0.0 ], [ 3.998553814899393, 51.987323088053131, 0.0 ], [ 3.987086024543889, 51.983489630873457, 0.0 ], [ 3.980367408521741, 51.978923864007278, 0.0 ], [ 3.972858821044363, 51.972254400764548, 0.0 ], [ 3.964975414140348, 51.962100235646908, 0.0 ], [ 3.965767078274776, 51.958192943818972, 0.0 ], [ 3.9714783688349, 51.946631494630573, 0.0 ], [ 3.983298387670107, 51.927838319363971, 0.0 ], [ 3.987657598176972, 51.919192330532567, 0.0 ], [ 3.996387468717351, 51.916655170259958, 0.0 ], [ 4.001520162387755, 51.917060309257401, 0.0 ], [ 4.012557173047844, 51.920229693122977, 0.0 ], [ 4.048001660981074, 51.929016004063037, 0.0 ], [ 4.074660383723856, 51.934902766449348, 0.0 ], [ 4.093265372959262, 51.934893193914142, 0.0 ], [ 4.109255167927888, 51.93372388247014, 0.0 ], [ 4.126734837924461, 51.932516710383318, 0.0 ], [ 4.139980260895813, 51.931607190411903, 0.0 ], [ 4.147852124552989, 51.929605717664337, 0.0 ], [ 4.154871093926527, 51.92643205114458, 0.0 ], [ 4.164068865894088, 51.922235472642527, 0.0 ], [ 4.168497166059588, 51.919961894548123, 0.0 ], [ 4.173765785457206, 51.917888566713337, 0.0 ], [ 4.177471853896762, 51.916075611730989, 0.0 ], [ 4.19143525340012, 51.910078064370317, 0.0 ], [ 4.202511571167557, 51.904617634287582, 0.0 ], [ 4.212615864448434, 51.899592844692521, 0.0 ], [ 4.219017538516361, 51.895881651452001, 0.0 ], [ 4.223832400198837, 51.890530353414427, 0.0 ], [ 4.226873738466466, 51.887326711590937, 0.0 ], [ 4.228951660169189, 51.883411780650917, 0.0 ], [ 4.230693981376918, 51.879498998972053, 0.0 ], [ 4.23245567320008, 51.876804618361149, 0.0 ], [ 4.235713735314597, 51.874394644882933, 0.0 ], [ 4.240860704028409, 51.871936856471507, 0.0 ], [ 4.246713039932697, 51.869742419056138, 0.0 ], [ 4.256500276768536, 51.866468683159347, 0.0 ], [ 4.263681165876234, 51.864338061762467, 0.0 ], [ 4.270856008579235, 51.864165082808469, 0.0 ], [ 4.277892120705451, 51.86439728390355, 0.0 ], [ 4.286849974208358, 51.8645882221536, 0.0 ], [ 4.292522161762808, 51.864277013800361, 0.0 ], [ 4.299811679854974, 51.864101583593218, 0.0 ], [ 4.311303788739909, 51.864116069612663, 0.0 ], [ 4.329734623050854, 51.864276123508454, 0.0 ], [ 4.331499517754396, 51.864024053521959, 0.0 ], [ 4.333688920564924, 51.862880975070347, 0.0 ], [ 4.336262605418339, 51.859939034098609, 0.0 ], [ 4.34137585560622, 51.854360268676722, 0.0 ], [ 4.351538393640837, 51.847648283939193, 0.0 ], [ 4.362976373927347, 51.843831785091027, 0.0 ], [ 4.370084445558557, 51.841582728647857, 0.0 ], [ 4.380089933091171, 51.840617401653859, 0.0 ], [ 4.391081067036453, 51.839898516477582, 0.0 ], [ 4.399478875843261, 51.840163345822617, 0.0 ], [ 4.408773767123155, 51.841091522022268, 0.0 ], [ 4.416943663050734, 51.840074266710687, 0.0 ], [ 4.423210503951149, 51.838579340029192, 0.0 ], [ 4.427503697165283, 51.836513860483983, 0.0 ], [ 4.425576770167599, 51.834095807664362, 0.0 ], [ 4.429533318178938, 51.832633476567118, 0.0 ], [ 4.430960005176178, 51.833741029897993, 0.0 ], [ 4.434887510360975, 51.833765267256481, 0.0 ], [ 4.43690383938546, 51.83312364379659, 0.0 ], [ 4.444075259200149, 51.831057027288622, 0.0 ], [ 4.455433085635423, 51.829280289923247, 0.0 ], [ 4.46771578321205, 51.829690776121659, 0.0 ], [ 4.478878685926199, 51.830795076878637, 0.0 ], [ 4.488164498900886, 51.832451656061437, 0.0 ], [ 4.497474736200315, 51.832277533876983, 0.0 ], [ 4.504006118328753, 51.831563991137351, 0.0 ], [ 4.512048063063212, 51.829681547431278, 0.0 ], [ 4.52023617846478, 51.827734951843553, 0.0 ], [ 4.527396173112097, 51.827099310549393, 0.0 ], [ 4.533930342872021, 51.828087046218492, 0.0 ], [ 4.54119648874229, 51.829604882537978, 0.0 ], [ 4.546648500960357, 51.829305402579081, 0.0 ], [ 4.552248982461713, 51.827676804936857, 0.0 ], [ 4.554747912727908, 51.824662856298893, 0.0 ], [ 4.557658515089615, 51.818037740135757, 0.0 ], [ 4.562338329955282, 51.812146737080482, 0.0 ], [ 4.567790268643788, 51.808872879868893, 0.0 ], [ 4.575257216434827, 51.80678730057928, 0.0 ], [ 4.58251573514719, 51.805521253772739, 0.0 ], [ 4.590037627731165, 51.805171699165371, 0.0 ], [ 4.597774284387466, 51.804055987532443, 0.0 ], [ 4.604313450900881, 51.802297307784549, 0.0 ], [ 4.613475866941057, 51.800235897440317, 0.0 ], [ 4.617341098435928, 51.798124369718323, 0.0 ], [ 4.618362378336296, 51.796181342994132, 0.0 ], [ 4.621252023873148, 51.790276378816401, 0.0 ], [ 4.62409979956345, 51.782635434416783, 0.0 ], [ 4.623941953031045, 51.77630929086952, 0.0 ], [ 4.623064754206448, 51.763389962802563, 0.0 ], [ 4.624433481891639, 51.757517082463593, 0.0 ], [ 4.627814435262847, 51.75075427110778, 0.0 ], [ 4.627879472881458, 51.743955186361127, 0.0 ], [ 4.626354092832785, 51.736179071181077, 0.0 ], [ 4.623290219595805, 51.726941768179799, 0.0 ], [ 4.619409050652274, 51.721750961942199, 0.0 ], [ 4.616120550381302, 51.716182645862659, 0.0 ], [ 4.609566079180749, 51.709477107202552, 0.0 ], [ 4.605050592100282, 51.705466495857721, 0.0 ], [ 4.600730591394282, 51.702608721228763, 0.0 ], [ 4.593484968687383, 51.700694538430596, 0.0 ], [ 4.578029217511245, 51.698311310158317, 0.0 ], [ 4.566507458151197, 51.695868601503108, 0.0 ], [ 4.556646021680121, 51.694506492441057, 0.0 ], [ 4.544885547002719, 51.692528879578767, 0.0 ], [ 4.546338018922178, 51.687750048814117, 0.0 ], [ 4.567420841100791, 51.69114986797571, 0.0 ], [ 4.568815806261195, 51.688246505711163, 0.0 ], [ 4.574402919128991, 51.673642115923442, 0.0 ], [ 4.582440212968, 51.675045873653353, 0.0 ], [ 4.603759922942611, 51.67909424032014, 0.0 ], [ 4.609000212678733, 51.682787850019203, 0.0 ], [ 4.61322770400086, 51.687589032555543, 0.0 ], [ 4.616346068304478, 51.688505385639402, 0.0 ], [ 4.618177114470288, 51.689083597445311, 0.0 ], [ 4.611503886862787, 51.700539553693048, 0.0 ], [ 4.618547235224733, 51.704758620916408, 0.0 ], [ 4.621188579458506, 51.70912483940171, 0.0 ], [ 4.625278654281832, 51.71853614447808, 0.0 ], [ 4.62895981688983, 51.726591934181187, 0.0 ], [ 4.631015273970891, 51.731345868647637, 0.0 ], [ 4.632270606035429, 51.73633287458577, 0.0 ], [ 4.634070893980669, 51.74383147071385, 0.0 ], [ 4.633874592083471, 51.752192752634713, 0.0 ], [ 4.630948981882912, 51.759574802772647, 0.0 ], [ 4.629120910421564, 51.765486142676011, 0.0 ], [ 4.629532349597874, 51.770814439938789, 0.0 ], [ 4.630894904718586, 51.77947979515762, 0.0 ], [ 4.630205119517427, 51.784593728340461, 0.0 ], [ 4.644155121160485, 51.784738066131723, 0.0 ], [ 4.64675789658717, 51.785521955122, 0.0 ], [ 4.648523080480585, 51.787655536083861, 0.0 ], [ 4.648380883747016, 51.791059131401887, 0.0 ], [ 4.648653816976214, 51.797693391205897, 0.0 ], [ 4.64901104999494, 51.806929426135078, 0.0 ], [ 4.649005053746082, 51.807578765213528, 0.0 ], [ 4.644316669744375, 51.808880483719882, 0.0 ], [ 4.641063113733887, 51.807100885321567, 0.0 ], [ 4.636305758391581, 51.805050274482483, 0.0 ], [ 4.631933508015191, 51.803574293533259, 0.0 ], [ 4.626968120311867, 51.802438675475052, 0.0 ], [ 4.620497311172034, 51.802272999154503, 0.0 ], [ 4.615951679076103, 51.803311401541073, 0.0 ], [ 4.606272307986039, 51.806284851480697, 0.0 ], [ 4.596159971591947, 51.808126778121583, 0.0 ], [ 4.586158031535925, 51.808844780132617, 0.0 ], [ 4.579447886128047, 51.810058867002901, 0.0 ], [ 4.573060189604023, 51.812295490473929, 0.0 ], [ 4.566012519971096, 51.815434547502242, 0.0 ], [ 4.563773566221736, 51.818687634757516, 0.0 ], [ 4.563201599006845, 51.821957279842117, 0.0 ], [ 4.562968089313304, 51.824543181240337, 0.0 ], [ 4.558540613429947, 51.82964006740454, 0.0 ], [ 4.552433393126092, 51.832316314254932, 0.0 ], [ 4.545014088655261, 51.834290234059893, 0.0 ], [ 4.537158070334188, 51.833393779336141, 0.0 ], [ 4.530038207031499, 51.832102384556208, 0.0 ], [ 4.526108059542169, 51.831813208714102, 0.0 ], [ 4.521717619046028, 51.832590425155978, 0.0 ], [ 4.516732521897168, 51.833960245101871, 0.0 ], [ 4.514236527391809, 51.834496902354807, 0.0 ], [ 4.509974165494395, 51.83553112330867, 0.0 ], [ 4.501820011750192, 51.837106988151767, 0.0 ], [ 4.495840125581275, 51.837258622122228, 0.0 ], [ 4.486864901019976, 51.837137748460393, 0.0 ], [ 4.478183135596048, 51.83618389210767, 0.0 ], [ 4.472439264345458, 51.835228495563783, 0.0 ], [ 4.461775016379061, 51.833908716258769, 0.0 ], [ 4.453922237117189, 51.833744056940958, 0.0 ], [ 4.445880993540623, 51.83521472745835, 0.0 ], [ 4.433867160146994, 51.838656519273393, 0.0 ], [ 4.42470273645797, 51.841440135427547, 0.0 ], [ 4.42153152386223, 51.842243381585881, 0.0 ], [ 4.412288631756962, 51.844837057595839, 0.0 ], [ 4.404628276058093, 51.845147332925059, 0.0 ], [ 4.394141940257916, 51.844056513242528, 0.0 ], [ 4.385970084449699, 51.84384058231597, 0.0 ], [ 4.38009310776012, 51.844281463545563, 0.0 ], [ 4.36989301637785, 51.845910018863457, 0.0 ], [ 4.360483758821787, 51.848313888858193, 0.0 ], [ 4.354349626537848, 51.851008172324264, 0.0 ], [ 4.347909504952052, 51.855404628831913, 0.0 ], [ 4.342408539850937, 51.86054970135617, 0.0 ], [ 4.337899232746361, 51.865294765331257, 0.0 ], [ 4.335433216951134, 51.869785988478348, 0.0 ], [ 4.333552618008243, 51.873809653870651, 0.0 ], [ 4.348428925913814, 51.875946272051308, 0.0 ], [ 4.369163929813908, 51.878317553006468, 0.0 ], [ 4.38839016518595, 51.87815521453539, 0.0 ], [ 4.401107220355129, 51.877058298559803, 0.0 ], [ 4.426040664379522, 51.873026806667077, 0.0 ], [ 4.437079597260853, 51.871386012190683, 0.0 ], [ 4.448943520990141, 51.871564030747429, 0.0 ], [ 4.454263046403941, 51.871932213390657, 0.0 ], [ 4.454612554075805, 51.876499560242912, 0.0 ], [ 4.461854063353627, 51.892501464714812, 0.0 ], [ 4.466785298039618, 51.894872117693943, 0.0 ], [ 4.472093734132137, 51.894358029456889, 0.0 ], [ 4.475597745139166, 51.893088291767661, 0.0 ], [ 4.495575011014354, 51.897022475135103, 0.0 ], [ 4.497482873128506, 51.904708775055248, 0.0 ], [ 4.483610341771596, 51.910345930077462, 0.0 ], [ 4.482760758326021, 51.909513132342788, 0.0 ], [ 4.481354888673367, 51.907659452044847, 0.0 ], [ 4.480171615622785, 51.906587418413928, 0.0 ], [ 4.478656007166948, 51.905614334220239, 0.0 ], [ 4.47420728963661, 51.903986716234108, 0.0 ], [ 4.468091888502901, 51.902252150225287, 0.0 ], [ 4.465003985162294, 51.907646878763657, 0.0 ], [ 4.462309529254382, 51.906989997256652, 0.0 ], [ 4.458607445683542, 51.904702185072551, 0.0 ], [ 4.450510265256744, 51.902876957622773, 0.0 ], [ 4.444600706587229, 51.903309513507153, 0.0 ], [ 4.440641527550353, 51.905721770934349, 0.0 ], [ 4.43802672176208, 51.907881366164169, 0.0 ], [ 4.432905776054799, 51.910555872287738, 0.0 ], [ 4.426112874198747, 51.912467834061907, 0.0 ], [ 4.422383819883729, 51.913168929583513, 0.0 ], [ 4.411878223585475, 51.913540129396402, 0.0 ], [ 4.409521680173807, 51.911005537418113, 0.0 ], [ 4.409980373031104, 51.907863380341098, 0.0 ], [ 4.410238903774557, 51.905760620712883, 0.0 ], [ 4.40792824029834, 51.905695231506868, 0.0 ], [ 4.408236980724604, 51.901856095046021, 0.0 ], [ 4.402375508953853, 51.900192780344398, 0.0 ], [ 4.398972492127681, 51.899588847278451, 0.0 ], [ 4.398510798786443, 51.904247434360137, 0.0 ], [ 4.397094621487874, 51.90646667642546, 0.0 ], [ 4.385614359534216, 51.907989085010612, 0.0 ], [ 4.378087964881123, 51.909686675033377, 0.0 ], [ 4.374128281325, 51.908694908241579, 0.0 ], [ 4.367852653269681, 51.908306138351691, 0.0 ], [ 4.363397244884773, 51.909430864627993, 0.0 ], [ 4.356519147274849, 51.906592072056007, 0.0 ], [ 4.349296405899398, 51.904046653826121, 0.0 ], [ 4.348244134194987, 51.903702932010567, 0.0 ], [ 4.349406111299523, 51.899790994706308, 0.0 ], [ 4.344160954128946, 51.898618840978919, 0.0 ], [ 4.335081568788706, 51.896868371347999, 0.0 ], [ 4.326200276202865, 51.896412521739443, 0.0 ], [ 4.32457076936686, 51.900817957716093, 0.0 ], [ 4.319738626071643, 51.900546337822448, 0.0 ], [ 4.307247475566158, 51.900885572136751, 0.0 ], [ 4.276804688291795, 51.907441861806603, 0.0 ], [ 4.260087950487823, 51.913551219808411, 0.0 ], [ 4.25162487299372, 51.91669563964313, 0.0 ], [ 4.248376913383156, 51.91449410685788, 0.0 ], [ 4.245717112143339, 51.915364621818703, 0.0 ], [ 4.243630834376155, 51.916354360761702, 0.0 ], [ 4.235899751919456, 51.921703697987638, 0.0 ], [ 4.228306802315593, 51.928844045846439, 0.0 ], [ 4.215768636252541, 51.935587344495282, 0.0 ], [ 4.199751571089035, 51.941361577730078, 0.0 ], [ 4.187845586291519, 51.946336059832333, 0.0 ], [ 4.174792346056993, 51.952774770268881, 0.0 ], [ 4.160900691414067, 51.960268474450572, 0.0 ], [ 4.143905642848278, 51.96883626854671, 0.0 ], [ 4.128162638085904, 51.9746278367287, 0.0 ], [ 4.125530375394987, 51.975262010141122, 0.0 ], [ 4.120117786508599, 51.977185994794318, 0.0 ], [ 4.116406216069988, 51.978663950477113, 0.0 ], [ 4.10945387129291, 51.981203138086038, 0.0 ], [ 4.093364376007981, 51.984944583465733, 0.0 ], [ 4.047046106288641, 51.994530949601149, 0.0 ] ] )


In [6]:
nodes = []
edges = []

node_names = []

for edge in FG.edges(data = True):
    node_1 = FG.nodes[edge[0]]
    node_2 = FG.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        node_names.append(edge[0])
        node_names.append(edge[1])
        edges.append(edge)

### Addition of nodes and edges at port entrance (artificial anchorage area)

In [7]:
FG = nx.DiGraph() #FG_new

for i in range(len(nodes)):
    node = nodes[i]
    node_name = node_names[i]
    FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

for edge in edges:
    FG.add_edge(edge[0], edge[1], Info = edge[2]) 

for i in range(3):
    FG.remove_edge(find_closest_edge(FG,shapely.geometry.Point([4.488648,51.908018]))[0][0],find_closest_edge(FG,shapely.geometry.Point([4.488648,51.908018]))[0][1])

FG.remove_node('8860852')
FG.remove_node('B4705_A')
FG.remove_node('B4705_B')
FG.remove_node('B45486_B')

#Node 1
node_name = '8866969B'
node["geometry"] = shapely.geometry.Point([FG.nodes['8866969']['geometry'].x,
                                           FG.nodes['8866969']['geometry'].y])
node['X'] = node["geometry"].x
node['Y'] = node["geometry"].y
FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

Info = FG.edges[('8862214', '8864217')]['Info']
Info['StartJunctionId'] = '8866969'
Info['EndJunctionId'] = '8866969B'
Info['Length'] = 0
Info['length'] = pyproj.Geod(ellps="WGS84").inv(FG.nodes['8866969']['geometry'].x,
                                                FG.nodes['8866969']['geometry'].y,
                                                FG.nodes['8866969B']['geometry'].x,
                                                FG.nodes['8866969B']['geometry'].y,)[2]
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y)) + ')'

FG.add_edge('8866969', '8866969B',Info=Info)


#Node 2
node_name = '8866969A'
node["geometry"] = shapely.geometry.Point([FG.nodes['8866969']['geometry'].x,
                                           FG.nodes['8866969']['geometry'].y])
node['X'] = node["geometry"].x
node['Y'] = node["geometry"].y
FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

Info = FG.edges[('8862214', '8864217')]['Info']
Info['StartJunctionId'] = '8866969B'
Info['EndJunctionId'] = '8866969A'
Info['Length'] = 0
Info['length'] = pyproj.Geod(ellps="WGS84").inv(FG.nodes['8866969B']['geometry'].x,
                                                FG.nodes['8866969B']['geometry'].y,
                                                FG.nodes['8866969A']['geometry'].x,
                                                FG.nodes['8866969A']['geometry'].y,)[2]
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969A']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969A']['geometry'].y)) + ')'

FG.add_edge('8866969B', '8866969A',Info=Info)

#Node 3
node_name = '8866969C'
node["geometry"] = shapely.geometry.Point([FG.nodes['8866969']['geometry'].x,
                                           FG.nodes['8866969']['geometry'].y])
node['X'] = node["geometry"].x
node['Y'] = node["geometry"].y
FG.add_node(node_name, name = str((node['X'], node['Y'])), Position = (node['X'], node['Y']), geometry = node["geometry"])

Info = FG.edges[('8862214', '8864217')]['Info']
Info['StartJunctionId'] = '8866969B'
Info['EndJunctionId'] = '8866969C'
Info['Length'] = 0
Info['length'] = pyproj.Geod(ellps="WGS84").inv(FG.nodes['8866969B']['geometry'].x,
                                                FG.nodes['8866969B']['geometry'].y,
                                                FG.nodes['8866969C']['geometry'].x,
                                                FG.nodes['8866969C']['geometry'].y,)[2]
Info['Wkt'] = ('LINESTRING (' + str(FG.nodes['8866969B']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969B']['geometry'].y) + ', ' +
                                str(FG.nodes['8866969C']['geometry'].x) + ', ' +
                                str(FG.nodes['8866969C']['geometry'].y)) + ')'

FG.add_edge('8866969B', '8866969C',Info=Info)


### Makes FIS-network suitable for two-way traffic

In [8]:
FG = FG.to_undirected() 
FG = FG.to_directed()

### Plot of resulting FIS-network of the Port of Rotterdam

In [9]:
m = folium.Map(location=[51.85, 4.4], zoom_start = 10, tiles="cartodbpositron")

for node in FG.nodes(data = True):
    points_x = list(node[1]["geometry"].coords.xy[0])
    points_y = list(node[1]["geometry"].coords.xy[1])
    
    point = []
    for i, _ in enumerate(points_x):
        point.append((points_y[i], points_x[i]))
    else:
        folium.Circle(point[0], radius=12,fill=True,fill_opacity=1,tooltip = node[0],popup =node[0]).add_to(m)

for edge in FG.edges(data = True):
    points_x = list(edge[2]["Info"]["geometry"].coords.xy[0])
    points_y = list(edge[2]["Info"]["geometry"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    else:
        popup = folium.Popup(width=500, height=300)
        folium.PolyLine(line, weight = 3, tooltip = [edge[0],edge[1]], popup = [edge[0],edge[1]]).add_to(m)

m

In [32]:
from osgeo.osr import SpatialReference, CoordinateTransformation
 
# Define the Rijksdriehoek projection system (EPSG 28992)
epsg28992 = SpatialReference()
epsg28992.ImportFromEPSG(28992)
 
# correct the towgs84
epsg28992.SetTOWGS84(565.237,50.0087,465.658,-0.406857,0.350733,-1.87035,4.0812)
 
# Define the wgs84 system (EPSG 4326)
epsg4326 = SpatialReference()
epsg4326.ImportFromEPSG(4326)
 
rd2latlon = CoordinateTransformation(epsg28992, epsg4326)
latlon2rd = CoordinateTransformation(epsg4326, epsg28992)

node_lats = []
node_lons = []
for node in FG.nodes:
    node_lats.append(FG.nodes[node]['geometry'].y)
    node_lons.append(FG.nodes[node]['geometry'].x)

In [51]:
df = pd.DataFrame()
df['wgs84_lon'] = node_lons
df['wgs84_lat'] = node_lats
node_xs = []
node_ys = []
node_zs = []
for node_lon in enumerate(node_lons):
    node_rds = (latlon2rd.TransformPoint(node_lats[node_lon[0]], node_lon[1]))
    node_xs.append(node_rds[0])
    node_ys.append(node_rds[1])
    node_zs.append(node_rds[2])
df['rd_x'] = node_xs
df['rd_y'] = node_ys
df['rd_z'] = node_zs
df

,wgs84_lon,wgs84_lat,rd_x,rd_y,rd_z
0,4.013855,51.957121,60603.135706,441857.057402,0.0
1,3.999617,51.960532,59631.879149,442255.173829,0.0
2,3.982661,51.940640,58423.797898,440064.682177,0.0
3,3.996645,51.980158,59469.427142,444442.276460,0.0
4,3.970024,51.955880,57588.031060,441776.942814,0.0
...,...,...,...,...,...
327,4.229267,51.891133,75291.554861,434258.292721,0.0
328,4.463515,51.907378,91438.669339,435834.682963,0.0
329,4.044707,51.990540,62792.300169,445535.135759,0.0
330,4.044707,51.990540,62792.300169,445535.135759,0.0


In [54]:
filepath = os.path.join(os.getcwd(),'network_nodes.xyz')
df.to_csv(filepath,columns=['rd_x','rd_y','rd_z'],header=False,index=False)

## Create Simulation

In [10]:
simulation_start = datetime.datetime.now()
sim = model.Simulation(simulation_start,FG)
env = sim.environment
duration = 5*14*24*3600 #seconds

## Generate infrastructure

In [11]:
turning_basin_1 = core.IsTurningBasin(env = env, name = 'Turning Basin 1', node = '8866999', length = 300)
origin_1 = core.IsOrigin(env = env, name = 'Origin')
anchorage_1 = core.IsAnchorage(env = env, name = 'Anchorage 1', node = '8866969C', typ = 'sea_going_vessels',max_capacity = 50)
terminal_1 = core.IsTerminal(env = env, name = 'Koole terminal',length = 700, jetty_locations = [100,200,300,400,500], jetty_lengths = [300,300,300,300,300], node_start = '8866999', node_end = '8866859', typ = 'jetty')

## Assign infrastructure to FIS network and create junctions and sections

In [12]:
FG.nodes['8866969A']["Origin"] = [origin_1]
FG.nodes['8866969C']["Anchorage"] = [anchorage_1]
FG.edges['8866999', '8866859']["Terminal"] = [terminal_1]
FG.nodes['8866999']["Turning Basin"] = [turning_basin_1]

#Manual procedure required for names 'anchorage_access' and 'harbour_basin_access', and type 'one-way_traffic' ??
nodes_to_be_removed = []
for node in FG.nodes:
    number_of_edges = 0
    for edge in FG.edges:
        if node == edge[0]:
            number_of_edges+=1
    if number_of_edges == 0:
        nodes_to_be_removed.append(node)

for node in nodes_to_be_removed:
    FG.remove_node(node)
        
for node in FG.nodes:
    number_of_edges = 0
    for edge in FG.edges:
        if node == edge[0]:
            number_of_edges+=1
    if number_of_edges != 2:
        name_list = []
        type_list = []
        for edges in range(number_of_edges):
            name_list.append('waterway_access')
            type_list.append('two-way_traffic')
        FG.nodes[node]["Junction"] = core.IsJunction(env = [], name = [], sections = [], typ = [])
        FG.nodes[node]['Junction'].name = name_list
        FG.nodes[node]['Junction'].type = type_list

junction_nodes = []
for node in list(FG.nodes):
    if 'Junction' in FG.nodes[node]:
        junction_nodes.append(node)
        
for node1 in junction_nodes:
    names = []
    sections = []
    types = []
    for node2 in junction_nodes:
        if node1 == node2:
            continue

        route = nx.dijkstra_path(FG, node1, node2)
        section = True
        for node in route[1:-1]:
            if 'Junction' in FG.nodes[node]:
                section = False
                break

        if section:
            sections.append([node1,node2])
            names.append(FG.nodes[node1]["Junction"].name[len(sections)-1])
            types.append(FG.nodes[node1]["Junction"].type[len(sections)-1])
    
    FG.nodes[node1]["Junction"] = [core.IsJunction(env = env, name = names, sections = sections, typ = types)]

## Vessel database and generaton

In [13]:
# List 8 vessels
vdf = pd.DataFrame()
vdf[0] = ['Small coaster 1','Small coaster 2','Coaster','Handysize','Tanker MR','Tanker LR1','Tanker LR2 1','Tanker LR2 2']
vdf[1] = [71,110,126,149,184,228,243,249] #length
vdf[2] = [10.1,13.5,19,22,27,32,42,46] #beam
vdf[3] = [4.6,5.6,8.7,10.25,11.7,12.4,13.95,15.4] # draught + FWA
vdf[4] = 0.5*vdf[3] #unloaded draught
vdf[5] = [17,17,17,17,17,17,17,17] #H_e free board empty- Update!
vdf[6] = vdf[5]-(vdf[3]-vdf[4]) #H_f free board loaded
vdf[7] = [60,60,60,60,60,60,60,60] #t_b in minutes - berthing time (assumed in 1hs)
vdf[8] = [15*60,16.7*60,16.7*60,18.3*60,18.3*60,18.3*60,18.3*60,18.3*60] #t_l loading + unloading time (in minutes)
vdf[9] = [0,0,0,0,0,0,0,0] #UKC (ship-dependent)
vdf[10] = [4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5] # vessel speed in m/s - check with S. de Jong thesis
# vdf[10] = [6,6,6,6,6,6,6,6] # vessel speed in m/s - own assumption
vdf[11] = [18*60*60,18*60*60,18*60*60,18*60*60,18*60*60,18*60*60,18*60*60,18*60*60] # max waiting time in seconds
vdf[12] = [0,0,0,0,0,0,0,0] #critical cross-current velocity in m/s (ship-dependent)
vdf.columns = ['type','L','B','T_f','T_e','H_e','H_f','t_b','t_l','ukc','v','max_waiting_time','max_cross_current']
vdf

,type,L,B,T_f,T_e,H_e,H_f,t_b,t_l,ukc,v,max_waiting_time,max_cross_current
0,Small coaster 1,71,10.1,4.60,2.300,17,14.700,60,900.0,0,4.5,64800,0
1,Small coaster 2,110,13.5,5.60,2.800,17,14.200,60,1002.0,0,4.5,64800,0
2,Coaster,126,19.0,8.70,4.350,17,12.650,60,1002.0,0,4.5,64800,0
3,Handysize,149,22.0,10.25,5.125,17,11.875,60,1098.0,0,4.5,64800,0
4,Tanker MR,184,27.0,11.70,5.850,17,11.150,60,1098.0,0,4.5,64800,0
5,Tanker LR1,228,32.0,12.40,6.200,17,10.800,60,1098.0,0,4.5,64800,0
6,Tanker LR2 1,243,42.0,13.95,6.975,17,10.025,60,1098.0,0,4.5,64800,0
7,Tanker LR2 2,249,46.0,15.40,7.700,17,9.300,60,1098.0,0,4.5,64800,0


In [14]:
## USE THIS CELL FOR SHIP GENERATOR WITH constant ARRIVAL RATE
Vessel = type('Vessel', 
              (core.Identifiable, core.Movable, core.Routeable, core.VesselProperties, core.ExtraMetadata), {})

generator_sea = model.VesselGenerator(Vessel,vdf,random_seed=3)

In [15]:
### USE THIS CELL FOR SHIP GENERATOR WITH constant ARRIVAL RATE
origin = '8866969A' #coasters should enter empty and leave full (export) --> UPDATE SOMEWHERE
destination = '8866859'
sim.add_vessels(vessel_generator = generator_sea, origin = origin, destination = destination, 
                arrival_distribution = (3600/(5.5*3600)), arrival_process = 'Uniform', fleet_distribution = [1,1,1,1,1,1,1,1])


## Assign hydrodynamic data, width, depth and MBL to network

In [ ]:
#define variables
depth = [[],[]]
width = [[],[]]
MBL = [[],[]]
water_level=[[],[]]
current_velocity = [[],[]]
current_direction = [[],[]]
time = np.arange(0,duration,60)

# depth according to MBL values, and waterway navigational width obtained from measuring on HavenKaart
MBL[1] = 16.4*np.ones(len(FG.nodes))
width[1] = np.zeros(len(FG.nodes))
depth[1] = MBL[1]

# load water level, velocity magnitude and direction time series to each node
for nodes in enumerate(FG.nodes):
    MBL[0].append(FG.nodes[nodes[1]]['geometry'])
    width[0].append(FG.nodes[nodes[1]]['geometry'])
    depth[0].append((FG.nodes[nodes[1]]['geometry']))
    water_level[0].append((FG.nodes[nodes[1]]['geometry']))
    water_level[1].append([[],[]])
    current_velocity[0].append((FG.nodes[nodes[1]]['geometry']))
    current_velocity[1].append([[],[]])
    current_direction[0].append((FG.nodes[nodes[1]]['geometry']))
    current_direction[1].append([[],[]])
    
#for col in enumerate(df_wlevel.columns[1:]): #load water level
for col in enumerate(FG.nodes):
    water_level[1][col[0]][0]=[x-df_wlevel[df_wlevel.columns[0]][0]+simulation_start.timestamp() for x in list(df_wlevel[df_wlevel.columns[0]])]
    water_level[1][col[0]][1]=list(df_wlevel['Node 8'])    
    
#for col in enumerate(df_VM.columns[1:]): #load velocity magnitude
for col in enumerate(FG.nodes):
    current_velocity[1][col[0]][0]=[x-df_VM[df_VM.columns[0]][0]+simulation_start.timestamp() for x in list(df_VM[df_VM.columns[0]])]
    current_velocity[1][col[0]][1]=list(df_VM['Node 8'])
        
#for col in enumerate(df_Van.columns[1:]): #load velocity direction
for col in enumerate(FG.nodes):
    current_direction[1][col[0]][0]=[x-df_Van[df_Van.columns[0]][0]+simulation_start.timestamp() for x in list(df_Van[df_Van.columns[0]])]
    current_direction[1][col[0]][1]=list(df_Van['Node 8'])

In [ ]:
core.NetworkProperties.append_data_to_nodes(FG,width,depth,MBL,water_level,current_velocity,current_direction)
knots = 0.51444444444444

## Assign properties of the vertical and horizontal tidal window policy to network

In [ ]:
class window_method(Enum):
    critical_cross_current = 'Critical cross-current'
    point_based = 'Point-based'
    
class vessel_characteristics(Enum):
    min_ge_Length = ['minLength','>=']
    min_gt_Length = ['minLength','>']
    max_le_Length = ['maxLength','<=']
    max_lt_Length = ['maxLength','<']
    min_ge_Draught = ['minDraught','>=']
    min_gt_Draught = ['minDraught','>']
    max_le_Draught = ['maxDraught','<=']
    max_lt_Draught = ['maxDraught','<']
    min_ge_Beam = ['minBeam','>=']
    min_gt_Beam = ['minBeam','>']
    max_le_Beam = ['maxBeam','<=']
    max_lt_Beam = ['maxBeam','<']
    min_ge_UKC = ['minUKC','>=']
    min_gt_UKC = ['minUKC','>']
    max_le_UKC = ['maxUKC','<=']
    max_lt_UKC = ['maxUKC','<']
    Type = ['Type','==']

class vessel_direction(Enum):
    inbound = 'inbound'
    outbound = 'outbound'
    
class vessel_type(Enum):
    GeneralCargo = 'GeneralCargo'
    LiquidBulk = 'LiquidBulk'
    Container = 'Container'
    DryBulk = 'DryBulk'
    MultiPurpose = 'MultiPurpose'
    Reefer = 'Reefer'
    RoRo = 'RoRo'
    Barge = 'Barge'
    
class accessibility(Enum):
    non_accessible = 0
    accessible = -1
    
class tidal_period(Enum):
    Flood = 'Flood'
    Ebb = 'Ebb'
    
class current_velocity_type(Enum):
    CurrentVelocity = 'Current velocity'
    LongitudinalCurrent = 'Longitudinal current'
    CrossCurrent = 'Cross-current'
    
@dataclass
class vessel_specifications:
    vessel_characteristics: dict #{item of vessel_characteristics class: user-defined value,...}
    vessel_method: str #string containing the operators between the vessel characteristics (symbolized by x): e.g. '(x and x) or x'
    vessel_direction: str #item of vessel_direction class

    def characteristic_dicts(self):
        characteristic_dicts = {}
        for characteristic in self.vessel_characteristics:
            characteristic_dict = {characteristic.value[0]: [characteristic.value[1],self.vessel_characteristics[characteristic]]}
            characteristic_dicts = characteristic_dicts | characteristic_dict
        return characteristic_dicts

@dataclass
class window_specifications:
    window_method: str #item of window_method class
    current_velocity_values: dict #{tidal_period.Flood.value: user-defined value or item from accessibility class,...}
    current_velocity_ranges: dict = dict #if window_method is point-based: {tidal_period.Ebb.value: user-defined value,...}

@dataclass
class vtw_window_specifications:
    ukc_s: dict #{tidal_period.Flood.value: user-defined value or item from accessibility class,...}
    ukc_p: dict #{tidal_period.Flood.value: user-defined value or item from accessibility class,...}
    fwa: dict #{tidal_period.Flood.value: user-defined value or item from accessibility class,...}

@dataclass
class vertical_tidal_window_input:
    vessel_specifications: vessel_specifications #class
    window_specifications: window_specifications #class     
        
@dataclass
class horizontal_tidal_window_input:
    vessel_specifications: vessel_specifications #class
    window_specifications: window_specifications #class     
    condition: dict #{'Origin':node, 'Destination': node}
    data: list #Calculated input: [node,]
        

### Vertical tidal window

In [ ]:
for node in FG.nodes:
    vertical_tidal_window_inputs = []

    vessel_specification = vessel_specifications({vessel_characteristics.min_ge_Draught: 0},
                                                  'x',vessel_direction.inbound.value)

    window_specification = vtw_window_specifications({'ukc_s': 0.0},
                                                     {'ukc_p': 0.1},
                                                     {'fwa': 0.025})
    
    vertical_tidal_window_inputs.append(vertical_tidal_window_input(vessel_specifications = vessel_specification,
                                                                    window_specifications = window_specification))

    vessel_specification = vessel_specifications({vessel_characteristics.min_ge_Draught: 0},
                                                  'x',vessel_direction.outbound.value)

    window_specification = vtw_window_specifications({'ukc_s': 0.0},
                                                     {'ukc_p': 0.1},
                                                     {'fwa': 0.025})

    vertical_tidal_window_inputs.append(vertical_tidal_window_input(vessel_specifications = vessel_specification,
                                                                    window_specifications = window_specification))

    core.NetworkProperties.append_vertical_tidal_restriction_to_network(FG,node,vertical_tidal_window_inputs)

### Horizontal tidal window

In [ ]:
horizontal_tidal_window_inputs = []

previous_nodes = ['8861716','8861674'] 
node = '8861158'
next_node = '8867547'

for previous_node in previous_nodes:
    #Inbound_Vessels_Condition1
    vessel_specification = vessel_specifications({vessel_characteristics.min_ge_Length: 180,
                                                  vessel_characteristics.min_ge_Draught: 11.4,
                                                  vessel_characteristics.max_lt_Draught: 14.3},
                                                  '(x and x and x)',vessel_direction.inbound.value)

    window_specification = window_specifications(window_method.critical_cross_current.value,
                                                 {tidal_period.Flood.value: 2*knots,tidal_period.Ebb.value: 2*knots})

    horizontal_tidal_window_inputs.append(horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                        window_specifications = window_specification,
                                                                        condition = {'Origin': previous_node, 'Destination': next_node},
                                                                        data = [node, current_velocity_type.CurrentVelocity.value]));

    #Inbound_Vessels_Condition2
    vessel_specification = vessel_specifications({vessel_characteristics.min_ge_Draught: 14.3},
                                                  'x',vessel_direction.inbound.value)

    window_specification = window_specifications(window_method.point_based.value,
                                                 {tidal_period.Flood.value: 0.5*knots,tidal_period.Ebb.value: accessibility.accessible.value},
                                                 {tidal_period.Flood.value: 0.3,tidal_period.Ebb.value:0})

    horizontal_tidal_window_inputs.append(horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                        window_specifications = window_specification,
                                                                        condition = {'Origin': previous_node, 'Destination': next_node},
                                                                        data = [node, current_velocity_type.CurrentVelocity.value]));

    #Outbound_Vessels_Condition1
    vessel_specification = vessel_specifications({vessel_characteristics.max_lt_Draught: 14.3},
                                                  'x',vessel_direction.outbound.value)

    window_specification = window_specifications(window_method.point_based.value,
                                                 {tidal_period.Flood.value: 2*knots,tidal_period.Ebb.value: accessibility.accessible.value},
                                                 {tidal_period.Flood.value: 0,tidal_period.Ebb.value:0})

    horizontal_tidal_window_inputs.append(horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                        window_specifications = window_specification,
                                                                        condition = {'Origin': next_node, 'Destination': previous_node},
                                                                        data = [node, current_velocity_type.CurrentVelocity.value]));

    #Outbound_Vessels_Condition2
    vessel_specification = vessel_specifications({vessel_characteristics.min_ge_Draught: 14.3},
                                                  'x',vessel_direction.outbound.value)

    window_specification = window_specifications(window_method.point_based.value,
                                                 {tidal_period.Flood.value: 0.5*knots,tidal_period.Ebb.value: accessibility.non_accessible.value},
                                                 {tidal_period.Flood.value: 0.3,tidal_period.Ebb.value:0})

    horizontal_tidal_window_inputs.append(horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                        window_specifications = window_specification,
                                                                        condition = {'Origin': next_node, 'Destination': previous_node},
                                                                        data = [node, current_velocity_type.CurrentVelocity.value]));

core.NetworkProperties.append_horizontal_tidal_restriction_to_network(FG,node,horizontal_tidal_window_inputs)

## Plot of the bathymetry of the network

In [ ]:
positions = {}
for nodes in FG.nodes:
    node = FG.nodes[nodes]
    positions[nodes] = (node['geometry'].x, node['geometry'].y)

edge_count = []
for edge in enumerate(FG.edges):
    edge_count.append(FG.edges[edge[1]]['Info']['Depth'][0])

colormap = cm.get_cmap('Blues', 256)
fig, ax = plt.subplots(figsize=(16, 16))
ax.axis('off')
ax = fig.add_axes([0, 0.4, 1, 0.3]);
nx.draw(FG, positions, node_size = 10, node_color ='k', with_labels = False, horizontalalignment = 'right', verticalalignment = 'bottom', edge_color = edge_count, edge_cmap = colormap, edge_vmin = 0, arrows = False, width= 4)
ax.set_aspect('equal','box')
cbar = fig.colorbar(cm.ScalarMappable(cmap=colormap), ax=ax, ticks=[0, 1])
cbar.ax.set_yticklabels(['shallow','deep'])  # vertically oriented colorbar
plt.title('Bathymetry of Port X',fontsize = 14, fontweight='bold')
plt.show()

## Initiation of the simulation

In [ ]:
t0 = timepy.time()
sim.run(duration = duration) # this statement runs the simulation
t1 = timepy.time()
total = t1-t0
print(total) #simulation time in seconds

## Output

In [ ]:
vessels = sim.environment.vessels #extract vessels (entitie) from environment. It collects info while it moves through the network. That info is stored in the log file. The log file has 
env = sim.environment #extract the environment itself

In [ ]:
vessels[0].route

### A logfile  (visual check)

In [ ]:
# pd.set_option('display.max_rows', 500)
vessel = vessels[0]
print(vessel.type)
df = pd.DataFrame.from_dict(vessel.log) #creates a data frame with all the info of vessels[0].
df[0:60]

### Some operations and functions

In [ ]:
def calculate_distance(orig, dest):
    wgs84 = pyproj.Geod(ellps='WGS84')
    
    distance = wgs84.inv(orig[0], orig[1], 
                         dest[0], dest[1])[2]
    
    return distance

vessel_path_x = []
vessel_path_t = []

list_of_nodes = []
list_of_nodes.extend(nx.dijkstra_path(FG,'8866969','8866859'))
list_of_nodes.extend(nx.dijkstra_path(FG,'8866969','8866969C'))

vessel_path_x = []
vessel_path_t = []
for v in range(0,len(vessels)-1):
    vessel_path_xt = []
    vessel_path_tt = []
    distance = 0
    direction = 1
    vessel_path_t0 = simulation_start.timestamp()
    vessel_path_xt.append(distance)
    vessel_path_tt.append(vessels[v].log["Timestamp"][0].timestamp()-vessel_path_t0)
    for t in enumerate(vessels[v].log["Message"]):
        if t[1].split()[0] == 'Sailing':
            #current_node = t[1].split()[3]
            next_node = str(t[1].split()[6])
        if direction == 1:
            distance += calculate_distance((vessels[v].log["Geometry"][t[0]-1].x,
                                            vessels[v].log['Geometry'][t[0]-1].y),
                                           (vessels[v].log["Geometry"][t[0]].x,
                                            vessels[v].log['Geometry'][t[0]].y))
        elif direction == -1:
            distance -= calculate_distance((vessels[v].log["Geometry"][t[0]-1].x,
                                            vessels[v].log['Geometry'][t[0]-1].y),
                                           (vessels[v].log["Geometry"][t[0]].x,
                                            vessels[v].log['Geometry'][t[0]].y))
        
        vessel_path_xt.append(distance)
        vessel_path_tt.append(vessels[v].log["Timestamp"][t[0]].timestamp()-vessel_path_t0)
        if 'Origin' in vessels[0].env.FG.nodes[next_node].keys():
            direction = 1
        if 'Anchorage' in vessels[0].env.FG.nodes[next_node].keys() or 'Deberthing stop' in t[1]:
            direction = -1
    vessel_path_x.append(vessel_path_xt)
    vessel_path_t.append(vessel_path_tt)

In [ ]:
def tidal_window_start_stop_xloc(time,signal,correction,roots,start,end):
    if roots != []:
        for root_start in enumerate(roots):
            if root_start[1] >= start:
                break
        for root_end in enumerate(roots):
            if root_end[1] >= end:
                break

        def find_nearest(array, value):
            array = np.asarray(array)
            idx = (np.abs(array - value)).argmin()
            return idx,array[idx]

        idx,x = find_nearest(time,root_start[1])
        ylim = correction

        booleanlist = []
        rootlist = []
        rootlist.append(start)

        if root_start != root_end:
            if x >= root_start[1]:
                if signal[idx] > ylim:
                    booleanlist.append(0)
                    booleanlist.append(1)
                    rootlist.append(root_start[1])
                else:
                    booleanlist.append(1)
                    booleanlist.append(0)
                    rootlist.append(root_start[1])
            else:
                if signal[idx] > ylim:
                    booleanlist.append(1)
                    booleanlist.append(0)
                    rootlist.append(root_start[1])
                else:
                    booleanlist.append(0)
                    booleanlist.append(1)
                    rootlist.append(root_start[1])

            if booleanlist[0] == 0:
                boolean = 1
            else:
                boolean = 0

            for root in roots[(root_start[0]+1):root_end[0]]:
                if boolean == 0:
                    boolean = 1
                elif boolean == 1:
                    boolean = 0
                booleanlist.append(boolean)
                rootlist.append(root)
        else:
            boolean = 1
            booleanlist.append(boolean)

        rootlist.append(end)
        if boolean == 0:
            booleanlist.append(1)
        elif boolean == 1:
            booleanlist.append(0) 
    return booleanlist,rootlist

def tidal_window_start_stop_polygon(booleanlist,rootlist,figylimits):
    xfill_lists = []
    yfill_lists = []
    for i in range(len(booleanlist)):
        xfill_list = []
        yfill_list = []
        if booleanlist[i] == 0 and i != len(booleanlist)-1:
            xfill_list = [rootlist[i],rootlist[i],rootlist[i+1],rootlist[i+1]]
            yfill_list = [figylimits[0],figylimits[1],figylimits[1],figylimits[0]]
            xfill_lists.append(xfill_list)
            yfill_lists.append(yfill_list)
        elif i != len(booleanlist)-1:
            xfill_list = [rootlist[i],rootlist[i],rootlist[i+1],rootlist[i+1]]
            yfill_list = [figylimits[0],figylimits[1],figylimits[1],figylimits[0]]
            xfill_lists.append(xfill_list)
            yfill_lists.append(yfill_list)
    return xfill_lists,yfill_lists

def colors_tidal_window_polygons(xfill_lists,yfill_lists,booleanlist):
    color = []
    for i in range(len(xfill_lists)):
        if booleanlist[i] == 0:
            color.append('red')
        elif booleanlist[i] == 2:
            color.append('darkred')
        else:
            color.append('g')
    return color        
        
def times_tidal_window(vertical_tidal_window_bools,vertical_tidal_window_roots,horizontal_tidal_window_bools,horizontal_tidal_window_roots): 
    list_of_times_vertical_tidal_window = []
    list_of_times_horizontal_tidal_windows = []

    for time in range(len(vertical_tidal_window_roots)):
        list_of_times_vertical_tidal_window.append([vertical_tidal_window_roots[time],vertical_tidal_window_bools[time]])
    for time in range(len(horizontal_tidal_window_roots)):
        list_of_times_horizontal_tidal_windows.append([horizontal_tidal_window_roots[time],horizontal_tidal_window_bools[time]])

    list_indexes = list(np.arange(0, len(list_of_times_horizontal_tidal_windows) + 1))
    times_tidal_window = []
    list_of_list_indexes = []

    for time in list_of_times_vertical_tidal_window:
        times_tidal_window.append(time)
        list_of_list_indexes.append(0)
    for time in list_of_times_horizontal_tidal_windows:
        times_tidal_window.append(time)
        list_of_list_indexes.append(1)

    list_of_list_indexes = [x for _, x in sorted(zip(times_tidal_window, list_of_list_indexes))]
    times_tidal_window.sort()
    
    indexes_to_be_removed = []
    for list_index in list_indexes:
        for time1 in range(len(times_tidal_window)):
            if times_tidal_window[time1][1] == 0 and list_of_list_indexes[time1] == list_index:
                for time2 in range(len(times_tidal_window)):
                    if time2 > time1 and times_tidal_window[time2][1] == 1 and list_of_list_indexes[time2] == list_index:
                        indexes = np.arange(time1 + 1, time2, 1)
                        for index in indexes:
                            indexes_to_be_removed.append(index)
                        break

    for time in range(len(times_tidal_window)):
        if time == 0:
            continue
        elif times_tidal_window[time][1] == 1 and times_tidal_window[time - 1][1] == 1:
            indexes_to_be_removed.append(time - 1)
        elif times_tidal_window[time][1] == 0 and times_tidal_window[time - 1][1] == 0:
            indexes_to_be_removed.append(time)

    indexes_to_be_removed.sort()
    indexes_to_be_removed = list(dict.fromkeys(indexes_to_be_removed))
      
    times_tidal_window_bools = []
    times_tidal_window_roots = []
    for time in times_tidal_window:
        times_tidal_window_bools.append(time[1])
        times_tidal_window_roots.append(time[0])
    
    return times_tidal_window_bools,times_tidal_window_roots,indexes_to_be_removed

def cross_current_calculation(magnitude,direction,origin,location,destination):
    origin_lat1 = FG.nodes[origin]['geometry'].x
    origin_lon1 = FG.nodes[origin]['geometry'].y
    destination_lat1 = FG.nodes[location]['geometry'].x
    destination_lon1 = FG.nodes[location]['geometry'].y
    fwd_azimuth1, _, _ = pyproj.Geod(ellps="WGS84").inv(origin_lat1, origin_lon1,destination_lat1, destination_lon1)
    origin_lat2 = FG.nodes[location]['geometry'].x
    origin_lon2 = FG.nodes[location]['geometry'].y
    destination_lat2 = FG.nodes[destination]['geometry'].x
    destination_lon2 = FG.nodes[destination]['geometry'].y
    fwd_azimuth2, _, _ = pyproj.Geod(ellps="WGS84").inv(origin_lat2, origin_lon2,destination_lat2, destination_lon2)
    cross_current_signal_at_node = []
    for t in range(len(magnitude)):
        cross_current_signal_at_node.append(np.max([abs(magnitude[t]*np.sin((direction[t]-fwd_azimuth2)/180*np.pi)),
                                                    abs(magnitude[t]*np.sin((direction[t]-fwd_azimuth1)/180*np.pi))]))
    return cross_current_signal_at_node

def color_vessels(vessel_type,vessel_types,color_list):
    for vtype in enumerate(vessel_types):
        if vessel_type == vtype[1]:
            color = color_list[vtype[0]]
    return color

def vessel_legend(axis,vessel_types,vessel_drafts,vessel_ukcs,color_list):
    for vtype in enumerate(vessel_types):
        axis.plot([0,0],[0,0],c=color_list[vtype[0]],label=str(vtype[1])+' ('+ str(round(vessel_drafts[vtype[0]]+vessel_ukcs[vtype[0]],2)) +' m)')
    axis.legend(loc='lower right',ncol=2)

### Time-distance diagram (visual check)

In [ ]:
terminal = vessels[0].env.FG.edges['8866999', '8866859']['Terminal'][0]
if terminal.type == 'quay':
    max_available_quay_length = terminal.length.capacity
if terminal.type == 'jetty':
    max_available_quay_length = 5
    
vesseltje = vessels[0]
start = 0
end =  duration
ax3xlist = [eta+depth[1][16] for eta in water_level[1][16][1]] #let's define it at node 15 to visualize when we have the critical bed level of MBL=-16.4m NAP
ax3ylist = [t-simulation_start.timestamp() for t in water_level[1][16][0]] 
ax4xlist = cross_current_calculation(current_velocity[1][14][1],current_direction[1][14][1],'8861716','8861158','8867547') # let's show the cross-current at the node 15 when the ship turns towards the access to 3ePH/Botlek area. This point is subsequent to Node 14: Scheurkade
ax4ylist = [t-simulation_start.timestamp() for t in current_velocity[1][14][0]]
required_water_level = vesseltje.metadata['ukc'] + vesseltje.T_f
root_interp_water_level_at_edge = sc.interpolate.CubicSpline(ax3ylist,[x-required_water_level for x in ax3xlist])
root_interp_cross_current_at_edge = sc.interpolate.CubicSpline(ax4ylist,[abs(x)-vessels[0].metadata['max_cross_current'] for x in ax4xlist])
if root_interp_water_level_at_edge.roots() != []:
    stst_vtw,roots_vtw = tidal_window_start_stop_xloc(ax3ylist,ax3xlist,required_water_level,root_interp_water_level_at_edge.roots(),start,end)
else:
    stst_vtw,roots_vtw = [],[]
if root_interp_cross_current_at_edge.roots() != []:
    stst_htw,roots_htw = tidal_window_start_stop_xloc(ax4ylist,ax4xlist,-vessels[0].metadata['max_cross_current'],root_interp_cross_current_at_edge.roots(),start,end)
else:
    stst_htw,roots_htw = [],[]
    
stst_tw,roots_tw,indexes_to_be_removed = times_tidal_window(stst_vtw,roots_vtw,stst_htw,roots_htw)
if len(roots_vtw) >= 2 and len(roots_htw) >= 2:
    for i in indexes_to_be_removed:
        if stst_tw[i] == 0:
            stst_tw[i] = 2   
        elif stst_tw[i] == 1:
            stst_tw[i] = 0
    
if terminal.type == 'quay':
    time_available_quay_length = []
    available_quay_length = []
    quay_level = 0
    time_available_quay_length.append(0)
    available_quay_length.append(quay_level)
    for t in range(len(terminal.log["Message"])):
        time_available_quay_length.append(terminal.log["Timestamp"][t].timestamp()-simulation_start.timestamp())
        available_quay_length.append(quay_level)
        time_available_quay_length.append(terminal.log["Timestamp"][t].timestamp()-simulation_start.timestamp())
        available_quay_length.append(terminal.log["Value"][t])
        quay_level = terminal.log["Value"][t]
        
if terminal.type == 'jetty':
    time_available_quay_length = []
    available_quay_length = []
    quay_level = 0
    time_available_quay_length.append(0)
    available_quay_length.append(quay_level)
    for t in range(len(terminal.log["Message"])):
        time_available_quay_length.append(terminal.log["Timestamp"][t].timestamp()-simulation_start.timestamp())
        available_quay_length.append(quay_level)
        time_available_quay_length.append(terminal.log["Timestamp"][t].timestamp()-simulation_start.timestamp())
        available_quay_length.append(terminal.log["Value"][t])
        quay_level = terminal.log["Value"][t]
    
anchorage = vessels[0].env.FG.nodes['8866969C']['Anchorage'][0]
time_anchorage_occupation = []
anchorage_occupation = []
anchorage_capacity = 0
time_anchorage_occupation.append(0)
anchorage_occupation.append(anchorage_capacity)
for t in range(len(anchorage.log["Message"])):
    time_anchorage_occupation.append(anchorage.log["Timestamp"][t].timestamp()-simulation_start.timestamp())
    anchorage_occupation.append(anchorage_capacity)
    time_anchorage_occupation.append(anchorage.log["Timestamp"][t].timestamp()-simulation_start.timestamp())
    anchorage_occupation.append(anchorage.log["Value"][t])
    anchorage_capacity = anchorage.log["Value"][t]

distance_to_anchorage = 0
distance_to_terminal = 25000
fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4,figsize=(16, 9),gridspec_kw={'width_ratios': [3.5, 1, 1, 1]})
ax1.set_xlim([0,distance_to_terminal])
figxlimits = ax1.axes.get_xlim()
xfill_lists,yfill_lists = tidal_window_start_stop_polygon(stst_tw,roots_tw,figxlimits)
colors = colors_tidal_window_polygons(yfill_lists,xfill_lists,stst_tw)
for c in range(len(colors)):
    ax1.fill(yfill_lists[c],xfill_lists[c],colors[c],alpha=.3)
vessel_types = list(vdf['type'])
vessel_drafts = list(vdf['T_f'])
vessel_ukcs = list(vdf['ukc'])
color_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
ax1.axvline(distance_to_anchorage,color = 'k',linestyle = '--')
ax1.axvline(distance_to_terminal,color = 'k',linestyle = '--')
ax1.text(distance_to_anchorage,1.02*end,'Anchorage',horizontalalignment  = 'center')
ax1.text(distance_to_terminal,1.02*end,'Terminal',horizontalalignment  = 'center')
vessel_types = list(vdf['type'])
for v in reversed(range(0,len(vessels)-1)):
    color = color_vessels(vessels[v].type,vessel_types,color_list)
    ax1.plot(vessel_path_x[v],vessel_path_t[v],c=color)
vessel_legend(ax1,vessel_types,vessel_drafts,vessel_ukcs,color_list)
ax1.set_title("Tracking diagram for vessels calling at port", fontweight='bold', pad = 42)
ax1.set_xlabel('Distance [m]')
ax1.set_xlim(figxlimits)
ax1.set_ylabel('Time [s]')
ax1.set_ylim([start,end]);

ax2.plot([max_available_quay_length-x for x in available_quay_length],time_available_quay_length)
ax2.set_xlim([0,1.1*max_available_quay_length])
figxlimits = ax2.axes.get_xlim()
xfill_lists,yfill_lists = tidal_window_start_stop_polygon(stst_tw,roots_tw,figxlimits)
colors = colors_tidal_window_polygons(yfill_lists,xfill_lists,stst_tw)
for c in range(len(colors)):
    ax2.fill(yfill_lists[c],xfill_lists[c],colors[c],alpha=.3)
if terminal.type == 'quay':
    ax2.axvline(vessels[0].L,color = 'k', linestyle = '--')
    ax2.set_title("Available quay length \n over time", fontweight='bold', pad = 32)
    ax2.text(vessels[0].L,1.01*end,'Required quay \n length',horizontalalignment = 'center') 
elif terminal.type == 'jetty':
    ax2.axvline(1,color = 'k', linestyle = '--')
    ax2.set_title("Available jetties \n over time", fontweight='bold', pad = 32)
ax2.set_xlim(figxlimits)
ax2.yaxis.set_visible(False)
ax2.set_ylim([start,end])
ax2.set_xlabel('Number of jetties [-]');

ax3.plot(ax3xlist,ax3ylist)
ax3.plot([eta+depth[1][6] for eta in water_level[1][6][1]],ax3ylist,c='#1f77b4')
ax3.set_ylim([start,end])
figxlimits = ax3.axes.get_xlim()
xfill_lists,yfill_lists = tidal_window_start_stop_polygon(stst_tw,roots_tw,figxlimits)
colors = colors_tidal_window_polygons(yfill_lists,xfill_lists,stst_tw)
for c in range(len(colors)):
    ax3.fill(yfill_lists[c],xfill_lists[c],colors[c],alpha=.3)
ax3.yaxis.set_visible(False)
ax3.set_ylim([start,end])
ax3.set_xlim(figxlimits)
ax3.set_xlabel('Water depth [m]');

if required_water_level >= ax3.axes.get_xlim()[1]:
    ax3.axvline(ax3.axes.get_xlim()[1],color = 'k', linestyle = '--')
    ax3.text(ax3.axes.get_xlim()[1],1.01*end,'Required water \n depth',horizontalalignment = 'center')
elif required_water_level <= ax3.axes.get_xlim()[0]:
    ax3.axvline(ax3.axes.get_xlim()[0],color = 'k', linestyle = '--')
    ax3.text(ax3.axes.get_xlim()[0],1.01*end,'Required water \n depth',horizontalalignment = 'center')
else:  
    ax3.axvline(required_water_level,color = 'k', linestyle = '--')
    ax3.text(required_water_level,1.01*end,'Required water \n depth',horizontalalignment = 'center')
ax4.yaxis.set_visible(False)
ax3.set_title("Available water depth \n over time", fontweight='bold', pad = 32)
ax3.yaxis.set_visible(False)
ax4.set_xlabel('Cross-current \n velocity [m/s]',horizontalalignment = 'center');

ax4.plot(ax4xlist,ax4ylist)
ax4.set_ylim([start,end])
figxlimits = ax4.axes.get_xlim()
xfill_lists,yfill_lists = tidal_window_start_stop_polygon(stst_tw,roots_tw,figxlimits)
colors = colors_tidal_window_polygons(yfill_lists,xfill_lists,stst_tw)
for c in range(len(colors)):
    ax4.fill(yfill_lists[c],xfill_lists[c],colors[c],alpha=.3)
ax4.set_title("Cross-current velocity \n over time", fontweight='bold', pad = 32)
ax4.set_ylim([start,end])
ax4.set_xlim(figxlimits)
if vessels[0].metadata['max_cross_current'] <= ax4.axes.get_xlim()[1]:
    ax4.axvline(vessels[0].metadata['max_cross_current'],color = 'k', linestyle = '--')
    ax4.text(vessels[0].metadata['max_cross_current'],1.01*end,'Critical cross \n current',horizontalalignment = 'center')
else:
    ax4.axvline(ax4.axes.get_xlim()[1],color = 'k', linestyle = '--')
    ax4.text(ax4.axes.get_xlim()[1],1.01*end,'Critical cross \n current',horizontalalignment = 'center')
ax4.yaxis.set_visible(False)
plt.show();